# *Template*: pyODV

---

## Overview
Ocean Data Viewer is a nifty piece of software for visualizing gridded data like the World Ocean Atlas. This notebook provides tips about how to make **surface** (constant depth), **section** (constant latitude or longitude), **profile** (constant longitude and latitude) plots. For each plot type there are specific instructions depending on whether latitude and longitude are vectors or arrays in your dataset.

---

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
import os
import pandas as pd
import seaborn as sns
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil

In [ ]:
lat_u = r'$^{\circ}$N'
lon_u = r'$^{\circ}$E'
depth_u = #string, e.g. 'm'
time_u = #string, e.g. 'ky BP'

surface_title = 'depth={} '+depth_u+ ', time={} '+time_u


## Load Data

In [ ]:
file_path = # path
ds = xr.open_dataset(file_path, use_cftime=True)
ds

In [ ]:
lat_coord = # string
lon_coord = # string
depth_coord = # string

In [ ]:
lon_dim = # string
lat_dim = # string

If your data has a time dimension, pick a snapshot of interest by specifying a time:

In [ ]:
time = # float

In [ ]:
snapshot_data = ds.sel(**{'time':time}, method='nearest')

## Surface: lon v. lat + color
A surface plot looks the geographic distribution of some variable on a plane. Here, we'll limit our area of interest to part of the Atlantic and pick a constant depth surface.

In [ ]:
def between(ds, var, lims):
    _ds = ds.where((ds.coords[var]<max(lims)) & (ds.coords[var]>min(lims)), drop=True)
    return _ds

In [ ]:
lonlims = # list of longitude bounds
lon_ds = between(snapshot_data, lon_coord, lonlims)

latlims = # list of latitude bounds
subarea_ds = between(lon_ds, lat_coord, latlims)


In [ ]:
var = # string
var_units = # string
depth = # float or int

In [ ]:
surf_ds = subarea_ds.sel(**{depth_coord:depth}, method='nearest')
snapshot_data_surf = surf_ds[var].squeeze()
snapshot_data_surf = c_snapshot_data_surf.compute()

Doesn't hurt to take a quick look a the distribution of the data. Particularly given we will want to use a colorbar, it's handy to check in on the range and density of values.

In [ ]:
bins =surf_ds[var].compute().plot.hist()
bin_edges = bins[1]

And now for the plot! There aren't any raging outliers, so we'll build our color mapping based on the full range of data.

In [ ]:
# establish scale
max_lim = bin_edges[-1]#c_snapshot_data.max()
min_lim = bin_edges[0]#c_snapshot_data.min()

In [ ]:
def make_scalar_mappable(lims, cmap, n=None):
    ax_norm = mpl.colors.Normalize(vmin=min(lims), vmax=max(lims), clip=False)
    if type(cmap)==list:
        if n is None:
            ax_cmap = mpl.colors.LinearSegmentedColormap.from_list("MyCmapName",cmap)
        else:
            ax_cmap = mpl.colors.LinearSegmentedColormap.from_list("MyCmapName",cmap, N=n)
    else:
        if n is None:
            ax_cmap = plt.get_cmap(cmap)
        else:    
            ax_cmap = plt.get_cmap(cmap, n)
    ax_sm = cm.ScalarMappable(norm=ax_norm, cmap=ax_cmap)
    return ax_sm

In [ ]:
n_levels= 20
ax2_levels = np.around(np.linspace(min_lim, max_lim, n_levels), decimals=4)
# make scalar mappable
ax2_sm = make_scalar_mappable([min_lim, max_lim],'cividis' , n_levels)
cf2_kwargs = {'cmap':ax2_sm.cmap,'levels':ax2_levels, 'norm' : ax2_sm.norm}

As discussed a bit in the notebook about the perils of model grids, there are a few ways to go about plotting spatial distributions. `pcolormesh` takes meshes of x and y values (in our case longitude and latitude) that underpin the data of interest (they do not need to fall on a regular rectangular grid) and interpolates between them.

### 2D lat and lon arrays
If the latitude and longitude values are not vectors, but rather arrays, use the cell below:

In [ ]:
fig = plt.figure(figsize=(8, 4))
# 1 row, 2 columns, .05 space between columns, 8:.3 ratio of left column to right column
gs = gridspec.GridSpec(1, 2, wspace=0.05, width_ratios=[8, .3])

# add subplot with specified map projection and coastlines (GeoAxes)
ax2 = fig.add_subplot(gs[0, 0], projection=ccrs.Robinson(central_longitude=0))

c_snapshot_data_surf.plot.pcolormesh(ax=ax2, 
                                   transform=ccrs.PlateCarree(), 
                                   x=lon_coord, y=lat_coord, 
                                   levels=ax2_levels, 
                                  cmap=cf2_kwargs['cmap'], 
                                  norm=cf2_kwargs['norm'], 
                                  add_colorbar=False);

ax2.coastlines(linewidth=.5)
ax2.add_feature(cfeature.LAND, zorder=14)
# uncomment to constrain map scope
# ax2.set_extent([lonlims[0]-15,lonlims[1]+15, latlims[0], latlims[1]], crs=ccrs.PlateCarree())

ax2_cb = fig.add_subplot(gs[0, 1])
cb2 = plt.colorbar(ax2_sm,cax=ax2_cb, orientation='vertical',label='{} [{}]'.format(var,var_units),
                   format=FormatStrFormatter('%g'))

ax2.set_title(surface_title.format(depth, time));

### 1D lat and lon vectors
If the latitude and longitude coordinates are vectors, the filled contour plot below is simpler:

In [ ]:
fig = plt.figure(figsize=(8, 4))
gs = gridspec.GridSpec(1, 2, wspace=0.05, width_ratios=[8, .3])

# add subplot with specified map projection and coastlines (GeoAxes)
ax2 = fig.add_subplot(gs[0, 0], projection=ccrs.Robinson(central_longitude=0))

# place data on coordinate system with continuous x axis (longitude axis)
tas_c, lonc = cutil.add_cyclic_point(c_snapshot_data_surf[var], c_snapshot_data_surf[lon_coord])
# plot contourf on ax2 (geosubplot)
cf2 = ax2.contourf(lonc,c_snapshot_data_surf[lat_coord],tas_c.squeeze(), levels=ax2_levels, 
                                  cmap=cf2_kwargs['cmap'], 
                                  norm=cf2_kwargs['norm'], 
                                   transform=ccrs.PlateCarree())

ax2.add_feature(cfeature.COASTLINE, edgecolor='k',linewidth=.5)
ax2.add_feature(cfeature.LAND, zorder=14)
# uncomment to constrain map scope
# ax2.set_extent([lonlims[0]-15,lonlims[1]+15, latlims[0], latlims[1]], crs=ccrs.PlateCarree())

ax2_cb = fig.add_subplot(gs[0, 1])
cb2 = plt.colorbar(ax2_sm,cax=ax2_cb, orientation='vertical',label='{} [{}]'.format(var,var_units),
                   format=FormatStrFormatter('%g'))

ax2.set_title(surface_title.format(depth, time));

## Section: lat or lon v. depth
A section plot is a depth transect in which the x axis reflects a distance metric (latitude or longitude in this case), the y axis is depth with some variable plotted as color.  Here we will plot the data as a scatterplot, and then as filled contour plot. The calculations involved in producing filled contour plots require the data be more regular. because we are selecting data along a line of longitude, we can describe our coordinates with a vector of latitude values and a vector of depth values--the true tenants of a grid! 

As a nice bit of convenience, we introduce an inset map that makes it easier to contextualize the depth transect.

In [ ]:
def make_inset_map(ax, lats, lons, central_lon=0,central_lat=0):
    axin = inset_axes(ax, width=.25*5, height=.25*5, loc="lower right", 
                 axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                 axes_kwargs=dict(projection=ccrs.Orthographic(central_latitude=central_lat, 
                                                               central_longitude=central_lon)))
                                                               # central_lon)))

    axin.coastlines(linewidth=.5)
    axin.add_feature(cfeature.LAND, zorder=14)
    axin.plot(lons, lats, transform=ccrs.PlateCarree())
    axin.set_global()


### 2D lat and lon arrays
If the latitude and longitude values are not vectors, but rather arrays, use the cell below:

In [ ]:
# if lat and lon are 2D arrays, specify longitude band
lims = # list of longitude limits e.g., [340, 342]
_sect_ds = between(subarea_ds, lon_coord, lims)
sect_ds = _sect_ds[var].squeeze()
sect_ds = sect_ds.compute()

lat_lims = [min(sect_ds[lat_coord].mean(dim='nlon')), max(sect_ds[lat_coord].mean(dim='nlon'))]
lon_lims = np.mean(lims)*np.ones(2)

In [ ]:
fig = plt.figure(figsize=(9,5));
gs = gridspec.GridSpec(1, 2, wspace=0.05, width_ratios=[8, .3])
ax = fig.add_subplot(gs[0, 0]);
ax.patch.set_facecolor('lightgray')

ax.contourf(sect_ds[lat_coord].mean(dim=lon_dim), sect_ds.z_t_m,
                     sect_ds.mean(dim=lon_dim).squeeze().data,10, origin='upper',
                         levels=ax2_levels,
                         cmap=ax2_sm.cmap, 
                        norm=ax2_sm.norm)


ylims = ax.get_ylim()
ax.set_ylim([ylims[1], ylims[0]])
ax.set_ylabel('Depth [m]')
ax.set_xlabel('Latitude [{}]'.format(lat_u))

ax2_cb = fig.add_subplot(gs[0, 1])
cb2 = plt.colorbar(ax2_sm,cax=ax2_cb, orientation='vertical',label='{} [{}]'.format(var, colorbar_units),
                   format=FormatStrFormatter('%g'))

make_inset_map(ax, lat_lims, lon_lims, central_lon=0,central_lat=0)
ax.set_title('{}{}, time={} {}'.format(341,lon_u, time, time_u));

### 1D lat and lon vectors
If the latitude and longitude coordinates are vectors, the filled contour plot below is simpler:

In [ ]:
# if lat and lon are vectors, specify longitude value
lon_val = # float
_sect_ds = subarea_ds.sel(**{lon_coord:lon_val}, method='nearest')

sect_ds = _sect_ds[var].squeeze()
sect_ds = sect_ds.compute()

lat_lims = [sect_ds[lat_coord].data.min(), sect_ds[lat_coord].data.max()]
lon_lims = lon_val*np.ones(2)

In [ ]:
fig = plt.figure(figsize=(9,5));
gs = gridspec.GridSpec(1, 2, wspace=0.05, width_ratios=[8, .3])
ax = fig.add_subplot(gs[0, 0]);
# background color
ax.patch.set_facecolor('lightgray')

# for scatter, because color var is an array, need to make a mesh version of x and y vars
lat_mesh, depth_mesh = np.meshgrid(sect_ds[lat_coord], sect_ds[depth_coord])
ax.scatter(lat_mesh,depth_mesh,
                     c=sect_ds.squeeze().data,
                         cmap=ax2_sm.cmap, 
                        norm=ax2_sm.norm)


ylims = ax.get_ylim()
ax.set_ylim([ylims[1], max([-100,ylims[0]])])
ax.set_ylabel('Depth [m]')
ax.set_xlabel('Latitude [{}]'.format(lat_u))

ax2_cb = fig.add_subplot(gs[0, 1])
cb2 = plt.colorbar(ax2_sm,cax=ax2_cb, orientation='vertical',label='{} [{}]'.format(var, colorbar_units),
                   format=FormatStrFormatter('%g'))

make_inset_map(ax, lat_lims, lon_lims, central_lon=0,central_lat=0)
ax.set_title('{}{}, time={} {}'.format(lon_val,lon_u, time, time_u));

## Profile: var 1 v. depth
Last plot type!  We are frequently interested in how some variable changes with depth. This is a little simpler on a regular grid when it is possible to query by dimensions. Instead, we'll get as close as we can and then take the mean latitude-wise and then longitude-wise so we are left with a depth profile. 

### 2D lat and lon arrays
If the latitude and longitude values are not vectors, but rather arrays, use the cell below:

In [ ]:
lims = # longitude limits of profile area e.g., [340, 342]
_sect_ds = between(snapshot_data, lon_coord, lims)
lims = # latitude limits of profile area e.g., [40, 42]
prof_reg = between(_sect_ds[var].squeeze(), lat_coord, lims)
prof_reg = prof_reg.compute()

In [ ]:
lat = prof_reg[lat_coord].mean(dim=lon_dim)
lon = prof_reg[lon_coord].mean(dim=lat_dim)
prof = prof_reg.mean(dim=lat_dim).mean(dim=lon_dim)


### 1D lat and lon vectors
If the latitude and longitude coordinates are vectors, the filled contour plot below is simpler:

In [ ]:
prof = snapshot_data.sel(**{lat_coord:lat_val, lon_coord:lon_val}, method='nearest')
prof = prof.compute()

In [ ]:
prof_df = pd.DataFrame({'values': prof.data, 'label': ['prof_1' for ik in range(len(prof.data))], 'depth': prof[depth_coord].data})

In [ ]:
fig = plt.figure(figsize=(3,6));
ax = fig.add_subplot();

ax.plot('values', 'depth', data=prof_df)
ax.invert_yaxis()
ax.set_ylabel('depth [m]')
ax.set_xlabel('{} [{}]'.format(var, colorbar_unit))

---